<center><font size="6"><b>Комп'ютерний практикум 4.

Препроцесінг даних</b></font></center>

>Завантажимо бібліотеки та дані

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
path='https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DA0101EN-SkillsNetwork/labs/Data%20files/automobileEDA.csv'
df = pd.read_csv(path)
df.head()

> Розглянемо інформацію про датасет

In [ ]:
df.info()

In [ ]:
# розмір датасету
df.shape

In [ ]:
# кількість унікальних значень для кожного стовпця
df.nunique()

##__Описова статистика__

Функція `describe` автоматично обчислює базову статистику для всіх неперервних змінних. Будь-які значення `NaN` автоматично пропускаються в цій статистиці.


* кількість спостережень
* середнє значення
* стандартне відхилення (std)
* мінімальне значення
* IQR (квартилі: 25%, 50%та 75%)
* максимальне значення

## Попередній візуальний аналіз даних

In [ ]:
df.describe()

In [ ]:
pip install seaborn

> імпортуємо бібліотеки `seaborn` для створення статистичної графіки та модуль бібліотеки`matplotlib` `pyplot`

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline 

##__Числові дані__

Числові дані представлені типом `int64` або `float64`

Для візуалізації числових даних можна використовувати діаграми розсіювання з підігнаними лініями для розуміння лінійного зв'язку між даними.

> Можна використовувати функцію `regplot` для побудови діаграми розсіювання та лінійної регресії пакету `seaborn`

In [ ]:
# Зв'язок між змінними "об'єм двигуна" та "ціна"
sns.regplot(x="engine-size", y="price", data=df)

>Для визначення кореляції між числовими змінними можна використовувати функцію `corr()`

In [ ]:
# Кореляційна матриця між факторами "об'єм двигуна" та "ціна" 
df[["engine-size", "price"]].corr()

##__Категоріальні змінні__
Це змінні, які описують "характеристику" одиниці даних і вибираються з невеликої групи категорій. 

Категоріальні змінні можуть мати тип `object` або `int64`. 

>Візуалізувати категоріальні змінні можна за допомогою діаграми `boxplots` пакету `seaborn` відносно числової (цільової) змінної.

In [ ]:
# трансмісія відносно ціни
sns.boxplot(x="drive-wheels", y="price", data=df)

> Якщо включити категоріальні змінні в описову статистику, то можна побачити 
* кількість спостережень
* кількість унікальних записів
* значення, яке найчастіше зустрічається
* частота такого  значення

In [ ]:
df.describe(include=['object'])

> Для підрахунку кількості унікальних значень певного фактору використовують функцію `value_counts`

In [ ]:
df['drive-wheels'].value_counts()

> Функція `groupby` групує дані за різними категоріями. Дані групуються на основі однієї або кількох змінних, а аналіз проводиться по окремих групах.


In [ ]:
#виділимо три фактора в новий датасет
df_group_one = df[['drive-wheels','body-style','price']]

In [ ]:
# групування по одній категоріальній змінній як середнє числової змінної 
grouped_test1 = df_group_one.groupby(['drive-wheels'],as_index=False).mean()
grouped_test1

In [ ]:
# групування по двом категоріальним змінним як середнє числової змінної 
grouped_test2 = df_group_one.groupby(['drive-wheels','body-style'],as_index=False).mean()
grouped_test2

> можна групувати у зведену таблицю, використовуючи функцію `pivot()` 

In [ ]:
#використаємо попереднє групування для зведеної таблиці
grouped_pivot = grouped_test2.pivot(index='drive-wheels',columns='body-style')
grouped_pivot

###__Кодування категоріальних змінних__
>Включати в модель якісну змінну типу `object` недоцільно, тому її треба інтерпретувати в "кількісний" варіант.
Є декілька підходів до кодування категоріальних змінних:

1. Співставлення кожній категорії певного числового значення (кодування міток)

<center><img src="https://miro.medium.com/max/772/1*QQe-4476Oy3_dI1vhb3dDg.png" width="400"></center>

> Таке кодування можливе за допомогою функції `LabelEncoder()` модуля `preprocessing` пакету `sklearn`

Проблема такого кодування полягає в тому, що ви створюєте порядок в значеннях фактору, там де його може не бути, виходом є інший варінт кодування:

2. Створюються штучні змінні, які відповідають певній категорії, і в результаті ми отримаємо не одне значення-код, а вектор з '0' та '1', що звісно обтяжує модель змінними, але не надає перевагу жодній категорії.

Такий вид кодування називають _наївним_ або _dummy-кодуванням_

<center><img src="https://miro.medium.com/max/3600/1*ggtP4a5YaRx6l09KQaYOnw.png" width="600"></center>

> Даний вид кодування забезпечується функцією `OneHotEncoder()` модуля `preprocessing` пакету `sklearn` або функцією `get_dummies()` пакету `pandas`

3. Можна співставляти кожній категорії кількість значень, які вона містить

4. Інші способі кодування



In [ ]:
# імпортуємо модуль preprocessing
from sklearn import preprocessing 

In [ ]:
le = preprocessing.LabelEncoder() # обираємо функцію LabelEncoder()
le.fit(df['drive-wheels']) # застосовуємо її до категоріального фактору 'drive-wheels'
df['drive-wheels']=le.transform(df['drive-wheels']) # заміняємо категоріальні значення на закодовані 



In [ ]:
df['drive-wheels']

> Застосуємо `OneHotEncoder()` до стовпця `'drive-wheels'`

In [ ]:
df1=df['drive-wheels'] # використаємо закодований на попередньому етапі фактор 'drive-wheels'
ohe = preprocessing.OneHotEncoder(sparse=False) # обираємо функцію OneHotEncoder()
df1=ohe.fit_transform(df1.values.reshape(-1,1)) # заміняємо значення стовпця на векторні значення 

In [ ]:
df1

> Застосуємо `get_dummies()` до фактору `'body-style'`


In [ ]:
ohe_dum= pd.get_dummies(df[['body-style']])

In [ ]:
ohe_dum

> Закодуємо категорії стовпця `'body-style'` кількістю їх значень

In [ ]:
df['body-style'].value_counts()

In [ ]:
df2=df['body-style'].map(df.groupby('body-style').size())

In [ ]:
df2

##__Визначення та обробка відсутніх значень (Missing Values)__

___Перетворення `?` на `NaN`___

для заміни А на В можна використати команду 
 <pre> .replace (A, B, inplace = True) </pre>


In [ ]:
#NAN значення формуються командою nan пакету numpy
df.replace("?", np.nan, inplace = True)

>Існує два методи виявлення відсутніх даних:

1. <b> .isnull () </b> 
2. <b> .notnull () </b>

На виході отримаємо логічне значення, яке вказує на те, що значення, передане в аргумент, фактично не містить даних.

In [ ]:
missing_data = df.isnull()
missing_data.head()

Порахуємо кількість пропущених значень

In [ ]:
for column in missing_data.columns.values.tolist():
    print(column)
    print (missing_data[column].value_counts())
    print("")    

Для візуального відображення пропущених значень можна скористатись функцією `heatmap()` подавши у якості аргументу функцыю визначення пропущених значень `df.isnull()`

In [ ]:
plt.figure(figsize=(7,6))
sns.heatmap(df.isnull(), cbar=False,cmap='viridis')
plt.show()

##__Усунення пропущених даних__

  1. Видалити дані <br>
        а. Видалити весь рядок <br>
        b. Видалити весь стовпець (стовпчик незначний для моделі, або містить 50% і білше пропущених значень)
  2. Замінити дані <br>
        а. Замінити на середнє,максимальне, мінімальне (для числових значень) <br>
        b. Замінити на найчастіш вживане (для категоріальних даних) <br>
        c. Замінити іншою функцією

In [ ]:
# Обчислимо середнє для фактору "stroke"
avg_stroke=df['stroke'].astype('float').mean(axis=0)
print("Середнє для stroke:", avg_stroke)

In [ ]:
# Замінимо NaN у факторі "stroke" на середнє 
df["stroke"].replace(np.nan, avg_stroke, inplace=True)

In [ ]:
# Перевіримо наявність пропущених значень в даному факторі
missing_data = df.isnull()
print (missing_data["stroke"].value_counts())

> якщо необхідно видалити рядок з NaN використовують команду
`dropna()`

> після цього краще "перезавантажити" індекси для коректного відтворення датасету за допомогою команди `reset_index(drop=True, inplace=True)`

In [ ]:
# видалимо ряжок з  NaN в стовбчику  "horsepower-binned" 
df.dropna(subset=["horsepower-binned"], axis=0, inplace=True)

# перезапишемо індекси, оскільки було одне видалення 
df.reset_index(drop=True, inplace=True)

In [ ]:
# Перевіримо наявність пропущених значень в даному факторі
missing_data = df.isnull()
print (missing_data["horsepower-binned"].value_counts())

##__Групуванн даних (Binning)__ 
> __Бінінг__ - це процес перетворення числових даних у категоріальні. Зручний для числових значень широкого діапазону. Вихідні значення даних, які потрапляють у заданий невеликий проміжок - бін, замінюються значенням, що є представником цього інтервалу, часто центральним значенням.
Виділяють:

* Групування з фіксованою шириною бінів
* Квантильне групування
* Інстинктивне групування 

> Групування можна здійснювати вручну, або за допомогою функції `LKBinsDiscretizer()` модуля `preprocessing` пакету `sklearn`




In [ ]:
from sklearn.preprocessing import KBinsDiscretizer  # імпортуємо функцію
est = KBinsDiscretizer(n_bins = 5, encode = 'ordinal', strategy='quantile') # розіб'ємо на 5 груп квантильним підходом
df['price_code'] = est.fit_transform(np.array(df['price']).reshape(-1,1)) # створимо нову колонку з відповідним кодуванням в датасеті
df[['price','price_code']] #вивидемо оригінальний та перетворений стовпець


## __Нормалізація та Стандартизація даних__

> __Нормалізація даних__ - це зміна значень числових стовпців у наборі даних до загальної шкали (певного інтервалу значень), без спотворення відмінностей у діапазонах значень. 

> __Стандартизація даних__ передбачає таке пертворення даних, що призводить до нормального розподілу

Часто використовують таке масштабування даних:
* MinMax
$$ X_iscal = \frac{X_i - min(X)}{max(X) - min(X)} $$
* Standart Scaler
$$ X_iscal = \frac{X_i - mean(X)}{std(X)} $$
* Max
$$ X_iscal = \frac{X_i}{max(X)} $$
* $L_1$ norm
$$ X_iscal = \frac{X_i}{\sum_{i=1}^{m}X_i} $$
* $L_2$ norm (Euclidean norm)
$$ X_iscal = \frac{X_i}{\sqrt{\sum_{i=1}^{m}X_i^2}} $$

>Для MinMax-ного масштабування, яке перетворює числові дані в проміжок $[0, 1]$, можна використовувати функцію `MinMaxScaler()` модуля `preprocessing` пакету `sklearn`

> Для стандартизації можна використовувати функцію  `StandardScaler()`модуля `preprocessing` пакету `sklearn`


Виділимо новий датасет із вихідного з певними числовими факторами

In [ ]:
  new_df=df[['wheel-base','length','width','height']]         
  new_df.head()

In [ ]:
# MimMax-масштабування
from sklearn.preprocessing import MinMaxScaler
MMS = MinMaxScaler()
new_df1 = MMS.fit_transform(new_df)

In [ ]:
# перетворюємо отриманий масив у фрейм
new_df_mms=pd.DataFrame({'wheel-base':new_df1[:,0],'length':new_df1[:,1],'width':new_df1[:,2],'height':new_df1[:,3]})
new_df_mms.head()

In [ ]:
new_df_mms.describe()

In [ ]:
# Стандартизація даних 
from sklearn.preprocessing import StandardScaler
SS = StandardScaler()
new_df1 = SS.fit_transform(new_df)

In [ ]:
# перетворюємо отриманий масив у фрейм
new_df_ss=pd.DataFrame({'wheel-base':new_df1[:,0],'length':new_df1[:,1],'width':new_df1[:,2],'height':new_df1[:,3]})
new_df_ss.head()

In [ ]:
new_df_ss.describe()

##<center>__Самостійні завдання__</center>

> Скопіювати блок самостійних завдань в окремий файл ***LastName_CP4.ipynb***

> Інсталюйте необхідні пакети бібліотек Python

### Завдання №1

Завантажте дані з ресурсу

URL = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'

In [ ]:
# МІСЦЕ ДЛЯ КОДУ

### Завдання №2

* Виведіть інформацію про датасет, описову статистику для кількісних змінних та для якісних змінних

* побудуйте графік регресії для змінних `Survived` та `Pclass`


In [ ]:
# МІСЦЕ ДЛЯ КОДУ

### Завдання №3

* побудуйте `heatmap()` для візуального відображення пропущених значень
* для кількісної змінної поповніть пропущені дані середнім значенням
* для категоріальної змінної поповніть пропущені значення значенням, яке зустрічається найчастіше

In [ ]:
# МІСЦЕ ДЛЯ КОДУ

### Завдання №4

Оберіть 4 категоріальних змінних та закодуйте:
* Змінну_1 -  за допомогою функції `LabelEncoder()`
* Змінну_2 -  за допомогою функції `get_dummies()`
* Змінну_3 -  за допомогою функції `OneHotEncoder()`
* Змінну_4 -  кількістю значень відповідної категорії


In [ ]:
# МІСЦЕ ДЛЯ КОДУ

### Завдання №5
Поділіть на 4 вікові групи змінну `Age`

In [ ]:
# МІСЦЕ ДЛЯ КОДУ

### Завдання №6

* Створіть новий датафрейм з вихідного тільки із числових значень
* Застосуйте $MinMax$-Нормування даних та збережіть в новий датафрейм, виведіть описову статистику
* Застосуйте $Standart Scaling$ даних та збережіть в новий датафрейм, виведіть описову статистику
* Застосуйте $L_1$-Нормування та збережіть в новий датафрейм, виведіть описову статистику


In [ ]:
# МІСЦЕ ДЛЯ КОДУ